<a href="https://colab.research.google.com/github/albertlis/Plant-Disease-Recognition/blob/main/Conv_EfficientNetB0_predictions_2_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import tensorflow as tf

drive.mount("/content/drive")
!unzip -q /content/drive/My\ Drive/PDR/Data/original.zip -d /content
!rm -r sample_data

Mounted at /content/drive


Constants

In [1]:
img_shape = (224, 224, 3)
e_net_out_shape = (7, 7, 1280)
nr_of_imgs = 49940
nr_of_val_imgs = 3862
batch_size = 64
nr_of_classes = 39
train_path = './original/train'
val_path = './original/val'
model_path = '/content/drive/My Drive/PDR/Results/models/ClassifierB0_2Layers.h5'
monitor = 'val_acc'
train_imgs_filename, train_labels_filename = 'imgs.npy', 'labels.npy'
val_imgs_filename, val_labels_filename = 'val_imgs.npy', 'val_labels.npy'
preds_dtype = 'float16'

Get predictions from Convolutional part to fast train classifier

In [4]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

e_net = EfficientNetB0(include_top=False, weights="imagenet", input_shape=img_shape)

def make_conv_predictions():
    gen = ImageDataGenerator(rotation_range=45, horizontal_flip=True, vertical_flip=True, rescale=1/255)
    datagen = gen.flow_from_directory(train_path, target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')
    val_datagen = gen.flow_from_directory(val_path, target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')

    imgs = np.lib.format.open_memmap(train_imgs_filename, dtype=preds_dtype, mode='w+', shape=((nr_of_imgs,) + e_net_out_shape))
    labels = np.lib.format.open_memmap(train_labels_filename, dtype='uint8', mode='w+', shape=(nr_of_imgs, nr_of_classes))
    val_imgs = np.lib.format.open_memmap(val_imgs_filename, dtype=preds_dtype, mode='w+', shape=((nr_of_val_imgs,) + e_net_out_shape))
    val_labels = np.lib.format.open_memmap(val_labels_filename, dtype='uint8', mode='w+', shape=(nr_of_val_imgs, nr_of_classes))

    for i, (imgs_batch, labels_batch) in enumerate(datagen):
        count = i * batch_size
        line = ' '
        if not i % 20 and i != 0:
            line = '\n'
        print(f'%5d{line}' %(count), end='')
        if count > nr_of_imgs:
            break
        predictions = e_net.predict(imgs_batch)
        imgs[count : count + batch_size] = predictions
        labels[count : count + batch_size] = labels_batch
    print()

    for i, (imgs_batch, labels_batch) in enumerate(val_datagen):
        count = i * batch_size
        line = ' '
        if not i % 20 and i != 0:
            line = '\n'
        print(f'%5d{line}' %(count), end='')
        if count > nr_of_val_imgs:
            break
        predictions = e_net.predict(imgs_batch)
        val_imgs[count : count + batch_size] = predictions
        val_labels[count : count + batch_size] = labels_batch
    print()
    
make_conv_predictions()

16711680/16705208 [==============================] - 0s 0us/step
Found 49940 images belonging to 39 classes.
Found 3862 images belonging to 39 classes.
    0    64   128   192   256   320   384   448   512   576   640   704   768   832   896   960  1024  1088  1152  1216  1280
 1344  1408  1472  1536  1600  1664  1728  1792  1856  1920  1984  2048  2112  2176  2240  2304  2368  2432  2496  2560
 2624  2688  2752  2816  2880  2944  3008  3072  3136  3200  3264  3328  3392  3456  3520  3584  3648  3712  3776  3840
 3904  3968  4032  4096  4160  4224  4288  4352  4416  4480  4544  4608  4672  4736  4800  4864  4928  4992  5056  5120
 5184  5248  5312  5376  5440  5504  5568  5632  5696  5760  5824  5888  5952  6016  6080  6144  6208  6272  6336  6400
 6464  6528  6592  6656  6720  6784  6848  6912  6976  7040  7104  7168  7232  7296  7360  7424  7488  7552  7616  7680
 7744  7808  7872  7936  8000  8064  8128  8192  8256  8320  8384  8448  8512  8576  8640  8704  8768  8832  8896  8960
 9

In [2]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, InputLayer, BatchNormalization, Dropout
import tensorflow.keras.callbacks as clb
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.constraints import max_norm
import numpy as np
import random

norm = max_norm(4)

def build_model():
    model = Sequential()
    
    model.add(InputLayer(input_shape=e_net_out_shape))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(659, kernel_constraint=norm, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(39, kernel_constraint=norm, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['acc'])
    return model

def np_array_memmap_gen(feat_path, label_path, batch_size=128, shuffle_array=True):
    while 1:
        x = np.load(feat_path, mmap_mode='r')
        y = np.load(label_path, mmap_mode='r')
        lst = [i for i in range(x.shape[0])]

        if shuffle_array:
            random.shuffle(lst)

        iters = len(lst) // batch_size + 1

        for i in range(iters):
            start = i * batch_size
            end = (i + 1) * batch_size
            yield (x[lst[start : end]], y[lst[start : end]])

callbacks = [
            clb.ReduceLROnPlateau(monitor=monitor, factor=0.1, min_lr=1e-7, patience=3, verbose=1),
            clb.EarlyStopping(monitor=monitor, patience=7, verbose=1),
            clb.ModelCheckpoint(monitor=monitor, filepath=model_path, save_best_only=True, verbose=1)
            ]

train_gen = np_array_memmap_gen(train_imgs_filename, train_labels_filename, batch_size=batch_size)
val_gen = np_array_memmap_gen(val_imgs_filename, val_labels_filename, batch_size=batch_size)

train_steps = nr_of_imgs // batch_size + 1
val_steps = nr_of_val_imgs // batch_size + 1

# model = build_model()

model = load_model(model_path)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(train_gen, epochs=100, validation_data=val_gen, callbacks=callbacks, verbose=1, 
                    steps_per_epoch=train_steps, validation_steps=val_steps)

Epoch 1/100
781/781 [==============================] - 195s 246ms/step - loss: 1.4347 - acc: 0.5740 - val_loss: 3.7302 - val_acc: 0.2452

Epoch 00001: val_acc improved from -inf to 0.24521, saving model to /content/drive/My Drive/PDR/Results/models/ClassifierB0_2Layers.h5
Epoch 2/100
781/781 [==============================] - 132s 170ms/step - loss: 1.4971 - acc: 0.5614 - val_loss: 1.9013 - val_acc: 0.4653

Epoch 00002: val_acc improved from 0.24521 to 0.46530, saving model to /content/drive/My Drive/PDR/Results/models/ClassifierB0_2Layers.h5
Epoch 3/100
781/781 [==============================] - 94s 121ms/step - loss: 1.5032 - acc: 0.5582 - val_loss: 2.2301 - val_acc: 0.3809

Epoch 00003: val_acc did not improve from 0.46530
Epoch 4/100
781/781 [==============================] - 20s 26ms/step - loss: 1.5171 - acc: 0.5528 - val_loss: 2.2870 - val_acc: 0.4117

Epoch 00004: val_acc did not improve from 0.46530
Epoch 5/100
781/781 [==============================] - 20s 26ms/step - loss: 1